In [2]:
import os
import numpy as np
import pandas as pd
txt_files = [f for f in os.listdir("data/interpretation_label") if f.startswith("omi-") and f.endswith(".txt")]
# 遍历每个文件，查看其数据形状
for txt_filename in txt_files:
    prefix = txt_filename.split(".")[0]
    train_data_path = os.path.join("data/processed", f"{prefix}_train.pkl")
    test_data_path = os.path.join("data/processed", f"{prefix}_test.pkl")
    test_labels_path = os.path.join("data/processed", f"{prefix}_test_label.pkl")
    
    train_data = pd.read_pickle(train_data_path)
    test_data = pd.read_pickle(test_data_path)
    test_labels = pd.read_pickle(test_labels_path)
    # print(f"test_data type: {type(test_data)},test_labels type: {type(test_labels)}")
    # 为什么是通过pd读的，但是shape是ndarray？ 回答：因为在读取时，pandas会自动将数据转换为numpy数组格式
    # 转换为DataFrame格式
    if isinstance(train_data, np.ndarray):
        train_data = pd.DataFrame(train_data)
    if isinstance(test_data, np.ndarray):
        test_data = pd.DataFrame(test_data)
    if isinstance(test_labels, np.ndarray):
        test_labels = pd.DataFrame(test_labels, columns=["label"])
    elif isinstance(test_labels, pd.Series):
        test_labels = test_labels.to_frame(name="label")

    # print(f"文件: {txt_filename},test.pkl大小: {test_data.shape},test_label.pkl: {test_labels.shape}")
    print(f"文件: {txt_filename},train.pkl大小: {train_data.shape},test.pkl大小: {test_data.shape},test_label.pkl: {test_labels.shape}")

文件: omi-1.txt,train.pkl大小: (8640, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-10.txt,train.pkl大小: (8640, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-11.txt,train.pkl大小: (8640, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-12.txt,train.pkl大小: (7291, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-2.txt,train.pkl大小: (8640, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-3.txt,train.pkl大小: (8640, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-4.txt,train.pkl大小: (8640, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-5.txt,train.pkl大小: (8640, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-6.txt,train.pkl大小: (8640, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-7.txt,train.pkl大小: (8640, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-8.txt,train.pkl大小: (8640, 19),test.pkl大小: (4320, 19),test_label.pkl: (4320, 1)
文件: omi-9.txt,train.pkl大小: (8640, 19),te

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict

# 配置参数
class Config:
    window_size = 30
    stride = 5  # 增大步长以减少计算量
    similarity_threshold = 0.6
    deepwalk_dim = 64
    num_walks = 10
    walk_length = 20
    anomaly_threshold = 0.5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. 数据加载和预处理工具函数
def load_data(base_path="data/processed"):
    data_files = {
        'train': 'omi-1_train.pkl',
        'test': 'omi-1_test.pkl',
        'test_label': 'omi-1_test_label.pkl'
    }
    return {k: pickle.load(open(os.path.join(base_path, v), 'rb')) for k, v in data_files.items()}

def parse_interpretation(file_path):
    anomalies = defaultdict(list)
    with open(file_path) as f:
        for line in f:
            if ':' not in line:
                continue
            range_part, sensors_part = line.strip().split(':')
            start_end = range_part.split('-')
            if len(start_end) != 2:
                continue
            start, end = map(int, start_end)
            sensors = list(map(int, sensors_part.split(',')))
            for ts in range(start, end+1):
                anomalies[ts] = sensors
    return dict(anomalies)

# 2. 滑动窗口处理
def create_windows(data, window_size, stride):
    windows = []
    n_samples, n_features = data.shape
    for i in range(0, n_samples - window_size + 1, stride):
        windows.append(data[i:i+window_size, :])
    return np.array(windows)

# 3. 图构建工具函数
def calculate_combined_similarity(window1, window2):
    flat1 = window1.flatten()
    flat2 = window2.flatten()
    
    # Pearson相关系数
    try:
        pearson_corr, _ = pearsonr(flat1, flat2)
    except:
        pearson_corr = 0
    
    # DTW距离 (加速计算)
    dtw_dist = dtw.distance_fast(flat1, flat2)
    dtw_sim = 1 / (1 + dtw_dist)
    
    return 0.7 * pearson_corr + 0.3 * dtw_sim

def build_adjacency_matrix(windows, threshold):
    n = len(windows)
    adj = np.zeros((n, n))
    
    # 只计算局部连接以节省时间
    for i in tqdm(range(n), desc="Building graph"):
        for j in range(max(0, i-50), min(n, i+50)):
            if i != j and adj[i,j] == 0:
                sim = calculate_combined_similarity(windows[i], windows[j])
                if sim > threshold:
                    adj[i,j] = sim
                    adj[j,i] = sim
    return adj

# 4. DeepWalk实现
def deepwalk(G, num_walks, walk_length, embedding_size):
    walks = []
    nodes = list(G.nodes())
    
    for _ in range(num_walks):
        np.random.shuffle(nodes)
        for node in nodes:
            walk = [node]
            while len(walk) < walk_length:
                curr = walk[-1]
                neighbors = list(G.neighbors(curr))
                if neighbors:
                    walk.append(np.random.choice(neighbors))
                else:
                    break
            walks.append([str(x) for x in walk])
    
    model = Word2Vec(
        walks, 
        vector_size=embedding_size, 
        window=5, 
        min_count=0, 
        sg=1, 
        workers=4,
        epochs=10
    )
    return model

# 5. 异常检测模型
class AnomalyDetector(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.net(x)

def train_model(embeddings, num_sensors, epochs=20):
    model = AnomalyDetector(
        input_dim=Config.deepwalk_dim,
        hidden_dim=128,
        output_dim=num_sensors
    ).to(Config.device)
    
    # 正常样本的标签是全0
    dummy_labels = torch.zeros((len(embeddings), num_sensors), device=Config.device)
    train_data = torch.FloatTensor(embeddings).to(Config.device)
    
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCELoss()
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(train_data)
        loss = criterion(outputs, dummy_labels)
        loss.backward()
        optimizer.step()
        
    return model

# 6. 评估函数
def evaluate_results(test_outputs, interpretation, stride):
    results = {
        'timestamp': [],
        'true_sensors': [],
        'pred_sensors': [],
        'correct': [],
        'precision': [],
        'recall': [],
        'f1': []
    }
    
    # 按时间戳统计
    all_timestamps = sorted(interpretation.keys())
    for ts in all_timestamps:
        window_idx = ts // stride
        if window_idx >= len(test_outputs):
            continue
            
        sensor_scores = test_outputs[window_idx]
        pred_sensors = np.where(sensor_scores > Config.anomaly_threshold)[0] + 1  # 传感器从1开始编号
        true_sensors = interpretation[ts]
        
        tp = len(set(true_sensors) & set(pred_sensors))
        fp = len(set(pred_sensors) - set(true_sensors))
        fn = len(set(true_sensors) - set(pred_sensors))
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        results['timestamp'].append(ts)
        results['true_sensors'].append(true_sensors)
        results['pred_sensors'].append(pred_sensors.tolist())
        results['correct'].append(tp)
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1'].append(f1)
    
    return pd.DataFrame(results)

# 主流程
def process_file(interpretation_file):
    print(f"\nProcessing {interpretation_file}...")
    
    # 1. 加载数据
    data = load_data()
    interpretation = parse_interpretation(os.path.join("data/interpretation_label", interpretation_file))
    
    # 2. 创建滑动窗口
    train_windows = create_windows(data['train'], Config.window_size, Config.stride)
    test_windows = create_windows(data['test'], Config.window_size, Config.stride)
    print(f"Train windows: {train_windows.shape}, Test windows: {test_windows.shape}")
    
    # 3. 构建图
    adj_matrix = build_adjacency_matrix(train_windows, Config.similarity_threshold)
    G = nx.from_numpy_array(adj_matrix)
    
    # 4. DeepWalk嵌入
    print("Running DeepWalk...")
    model = deepwalk(G, Config.num_walks, Config.walk_length, Config.deepwalk_dim)
    embeddings = np.array([model.wv[str(i)] for i in range(len(train_windows))])
    
    # 5. 训练异常检测模型
    print("Training anomaly detector...")
    detector = train_model(embeddings, data['train'].shape[1])
    
    # 6. 测试集处理
    test_embeddings = []
    for window in tqdm(test_windows, desc="Processing test windows"):
        # 简化版: 使用最近邻
        similarities = [calculate_combined_similarity(window, train_windows[i]) 
                       for i in range(min(1000, len(train_windows)))]  # 限制数量加速计算
        most_similar = np.argmax(similarities)
        test_embeddings.append(embeddings[most_similar])
    test_embeddings = np.array(test_embeddings)
    
    # 7. 预测
    with torch.no_grad():
        test_outputs = detector(torch.FloatTensor(test_embeddings).to(Config.device)).cpu().numpy()
    
    # 8. 评估
    results_df = evaluate_results(test_outputs, interpretation, Config.stride)
    
    # 计算整体指标
    avg_precision = np.mean(results_df['precision'])
    avg_recall = np.mean(results_df['recall'])
    avg_f1 = np.mean(results_df['f1'])
    
    print(f"\nEvaluation for {interpretation_file}:")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1: {avg_f1:.4f}")
    
    # 保存详细结果
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)
    result_file = os.path.join(output_dir, f"results_{interpretation_file.replace('.txt', '.csv')}")
    results_df.to_csv(result_file, index=False)
    print(f"Detailed results saved to {result_file}")
    
    return {
        'file': interpretation_file,
        'avg_precision': avg_precision,
        'avg_recall': avg_recall,
        'avg_f1': avg_f1
    }

# 主函数
def main():
    # 获取所有解释标签文件
    txt_files = [f for f in os.listdir("data/interpretation_label") 
                 if f.startswith("omi-") and f.endswith(".txt")]
    
    if not txt_files:
        print("No interpretation files found in data/interpretation_label/")
        return
    
    # 处理所有文件
    summary_results = []
    for file in txt_files:
        try:
            result = process_file(file)
            summary_results.append(result)
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")
    
    # 输出汇总结果
    print("\n=== Summary Results ===")
    summary_df = pd.DataFrame(summary_results)
    print(summary_df)
    
    # 保存汇总结果
    if not os.path.exists("results"):
        os.makedirs("results")
    summary_df.to_csv("results/summary_results.csv", index=False)
    print("\nAll processing completed. Summary results saved to results/summary_results.csv")

if __name__ == "__main__":
    main()